In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import tqdm

In [10]:
class DeepQNetwork(nn.Module):
    def __init__(self):
        self.super()
        self.hidden1 = nn.Linear(18, 100)
        self.hidden2 = nn.Linear(100, 100)
        self.output = nn.Linear(100, 9)
    def forward(self, x): 
        x = x.view(-1,1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        return self.output(x)
    def set_player_name(name):
        self.player_name = name

In [ ]:
Transition = namedtuple('Transition', 
                        ('state', 'action', 'next_state', 'reward'))
 
class ReplayBuffer(object): 
 
    def __init__(self, capacity): 
        self.memory = deque([],maxlen=capacity) 
 
    def push(self, *args): 
        """Save a transition"""
        self.memory.append(Transition(*args)) 
 
    def sample(self, batch_size): 
        return random.sample(self.memory, batch_size) 
 
    def __len__(self): 
        return len(self.memory)

In [ ]:
class DeepQTraining():
    def __init__(self):
        self.DISCOUNT_FACTOR = 0.99
        self.BUFFER_SIZE = 10000
        self.BATCH_SIZE = 64
        self.TARGET_NET_UPDATE_STEP = 500
        self.LEARNING_RATE = 5e-4
        self.NB_GAMES = 20000
        self.MAX_GAME_LENGTH = 9
        self.turns = np.array(['X','O'])
        self.N_ACTIONS = 9
        
        
    def reset_parameters(self):
        self.buffer = ReplayBuffer(self.BUFFER_SIZE)
        self.agent1 = DeepQNetwork()
        self.agent2 = DeepQNetwork()
        self.optim = optim.Adam(self.agent1.parameters(), lr=self.LEARNING_RATE)
        self.criterion = nn.HuberLoss()
        self.env = TictactoeEnv()
    
    def get_other_player(self, model):
        return "X" if model.player_name == "O" else "O"
    
    def choose_action(self, epsilon, env, model):
        if random.random() <= epsilon:
            return torch.tensor(random.randrange(self.N_ACTIONS))
        else:
            state = grid_to_state(env, model)
            return model(state).max() # potential problem

    def model_optimiser(self):
        if len(self.buffer) < self.BATCH_SIZE: 
            return 
        
        samples = self.buffer.sample(self.BATCH_SIZE)
        #TODO 
        
        output = self.agent1(samples)
        loss = self.criterion(output, 
        
    def grid_to_state(self, env,  model):
        return torch.tensor([env.grid==env.player2value(model.player_name), grid==env.player2value(self.get_other_player(model.player_name))]) # Might be broken
        
    def simulate_game(self, env, opt_player, learner, epsilon_greedy):
        for m in range(self.MAX_GAME_LENGTH):
            if env.current_player == opt_player.player_name:
                move = opt_player.act(env.grid)
            else:
                move = self.choose_action(epsilon_greedy, env.grid)
            
            prev_grid = env.grid.copy()
            round_player = env.current_player
            
            if env.check_valid(move):
                _, end, winner = env.step(move)
                reward = env.reward(round_player)
            else:
                end = True
                reward = -1
                    
            if learner.player_name == round_player:
                self.buffer.push(grid_to_state(prev_grid), move, grid_to_state(env.grid.copy()), reward)
                
            if end:
                return 
                    
    def train(self, epsilon_greedy, adversary_epsilon):
        self.reset_parameters()
        
        for e in tqdm(range(NB_GAMES)):
            self.env.reset()
            self.agent1.player_name = turns[(e+1)%2]
            opt_player = OptimalPlayer(adversary_epsilon, player_name=turns[e%2])
            simulate_game(self.env, opt_player, self.agent1, epsilon_greedy)
                                          
            if (e+1)%self.TARGET_NET_UPDATE_STEP == 0:
                model_optimiser()